### download the required packages

In [ ]:
!pip3 install -r requirements.txt
!pip install onnxruntime==1.12.0
!pip3 install cython; pip3 install 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
!pip3 install cython_bbox
!pip3 install faiss-cpu
!pip3 install faiss-gpu
!python3 setup.py develop
!pip install loguru
!pip install thop
!pip install lap
!pip install yacs

### run the bot-sort  re-identification model
- For the appearance extractor, use the SBS-S50 fine-tuned on MOT17
- use the default parameters of MOT17

In [ ]:
!python3 tools/track.py "/content/drive/My Drive/visML_project/streetaware_sample_chase_1_sensor_1_right" --default-parameters --fast-reid-weights pretrained/mot17_sbs_S50.pth --benchmark "MOT17" --eval "test" --fp16 --fuse

### Draw the detection outputs on the corresponding frame
- the Bot-Sort output is a txt file where each line has the format: \<frame_id>, \<identity number>, \<bounding box left> , \<bounding box top>, \<bounding box width>, \<bounding box height>, and \<confidence score>

In [ ]:
import cv2
import numpy as np
import os

# Load tracking data
track_file = '/content/drive/My Drive/visML_project/outputs/botsort/chase_1/sensor_1/right_people_track.txt'
tracks = {}
with open(track_file, 'r') as file:
    for line in file:
        frame_id, track_id, x, y, w, h, conf, _, _, _ = map(float, line.strip().split(','))
        if frame_id not in tracks:
            tracks[frame_id] = []
        tracks[frame_id].append((track_id, x, y, w, h, conf))

# Assign a unique color to each track_id
color_map = {}
for frame in tracks.values():
    for track_id, *_ in frame:
        if track_id not in color_map:
            color_map[track_id] = (np.random.randint(0, 255), np.random.randint(0, 255), np.random.randint(0, 255))

# Directory containing frames
frames_dir = '/content/drive/My Drive/visML_project/streetaware_sample_chase_1_sensor_1_right/test/MOT17-01-FRCNN/img1/'

# Output directory
output_dir = '/content/drive/My Drive/visML_project/output_frames/sensor_2_left'

# Process each frame
for frame_file in sorted(os.listdir(frames_dir)):
    frame_path = os.path.join(frames_dir, frame_file)
    frame = cv2.imread(frame_path)
    if frame is None:
        continue

    frame_id = int(frame_file.split('.')[0])
    if frame_id in tracks:
        for track_id, x, y, w, h, conf in tracks[frame_id]:
            color = color_map[track_id]
            cv2.rectangle(frame, (int(x), int(y)), (int(x + w), int(y + h)), color, 2)
            cv2.putText(frame, f'ID: {int(track_id)}, Conf: {conf:.2f}', (int(x), int(y) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

    # Save the frame
    output_path = os.path.join(output_dir, frame_file)
    cv2.imwrite(output_path, frame)


### Code to visualize the bounding box and confidence score of the original detection file

In [2]:
# load the file:
json_file_path = 'StreetAware/sensor_1/left_people.json'
with open(json_file_path, 'r') as file:
    data = json.load(file)

In [ ]:
json_file_path = 'StreetAware/sensor_1/left_people.json'
with open(json_file_path, 'r') as file:
    data = json.load(file)import cv2
import os
import json
from google.colab.patches import cv2_imshow

# Load the JSON data
with open('/content/drive/My Drive/visML_project/StreetAware/sensor_1/left_people.json', 'r') as file:
    data = json.load(file)

# Directory containing frames
frames_dir = '/content/drive/My Drive/visML_project/StreetAware/sensor_1/img_left/'

# Output directory
output_dir = '/content/drive/My Drive/visML_project/output_frames'
os.makedirs(output_dir, exist_ok=True)

# Initialize counter for non-null frames
non_null_counter = 0
max_non_null_frames = 10

# Scaling factor
scale_factor = 0.5

# Process each item in the JSON data
for frame_index, frame_data in enumerate(data):
    if frame_data is None:
        continue

    # Increment the counter and break if maximum frames are processed
    non_null_counter += 1
    if non_null_counter > max_non_null_frames:
        break

    frame_file = f'{frame_index + 1:06d}.jpg'
    frame_path = os.path.join(frames_dir, frame_file)
    frame = cv2.imread(frame_path)

    if frame is None:
        print(f"Frame {frame_file} not found.")
        continue

    # Resize the frame
    frame = cv2.resize(frame, None, fx=scale_factor, fy=scale_factor, interpolation=cv2.INTER_AREA)

    # Draw bounding boxes and display confidence
    scores = frame_data.get('scores', [])
    for box_index, box in enumerate(frame_data['boxes']):
        top_left = (int(box[0][0] * scale_factor), int(box[0][1] * scale_factor))
        bottom_right = (int(box[1][0] * scale_factor), int(box[1][1] * scale_factor))

        # Determine the color based on the availability of a score
        if box_index < len(scores):
            color = (0, 255, 0)
            score = scores[box_index]
            text = f'{score:.2f}'
        else:
            color = (0, 0, 255)
            text = 'N/A'

        cv2.rectangle(frame, top_left, bottom_right, color, 2)
        cv2.putText(frame, text, (top_left[0], top_left[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

    # Display the frame
    output_path = os.path.join(output_dir, frame_file)
    cv2.imwrite(output_path, frame)
    cv2_imshow(frame)
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()


Output hidden; open in https://colab.research.google.com to view.

### Code to convert the frames to a video

In [ ]:
import cv2
import os

# Directory containing frames
frames_dir = 'output_frames/sensor_2_left/'

# Output directory
output_dir = 'output_videos/'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

output_video_path = os.path.join(output_dir, 'sensor_2_left.mp4')
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
fps = 30

first_frame_path = os.path.join(frames_dir, '000001.jpg')
first_frame = cv2.imread(first_frame_path)
frame_height, frame_width = first_frame.shape[:2]
video_writer = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

# Process each frame
sorted_frame_files = sorted(os.listdir(frames_dir))
for frame_file in sorted_frame_files:
    if frame_file.endswith('.jpg'):
        frame_path = os.path.join(frames_dir, frame_file)
        frame = cv2.imread(frame_path)
        if frame is None:
            continue
        video_writer.write(frame)

video_writer.release()
print(f"Video saved to {output_video_path}")


Video saved to output_videos/sensor_2_left.mp4


### Calculate the evaluation metrics and save to a json file

In [ ]:
def box_overlap_count(boxes):
    def overlap(box1, box2):
        # Check if box1 overlaps with box2
        x1_min, y1_min, x1_max, y1_max = box1
        x2_min, y2_min, x2_max, y2_max = box2
        return not (x1_max < x2_min or x2_max < x1_min or y1_max < y2_min or y2_max < y1_min)

    count = 0
    for i in range(len(boxes)):
        for j in range(i + 1, len(boxes)):
            if overlap(boxes[i], boxes[j]):
                count += 1
    return count


In [ ]:
import json

video_name = "streetaware_sample_chase_2_sensor_2_right"
txt_path = 'outputs/botsort/chase_2/sensor_2/right_people_track.txt'
output_json = 'outputs/botsort/chase_2/sensor_2/right_people_track.json'
frame_data = {}

# Reading data from file
with open(txt_path, 'r') as file:
    data = file.read()

for line in data.strip().split("\n"):
    parts = line.split(',')
    frame_id = int(parts[0])
    people_id = int(parts[1])
    x1 = float(parts[2])
    y1 = float(parts[3])
    width = float(parts[4])
    height = float(parts[5])
    conf = float(parts[6])
    x2 = x1 + width
    y2 = y1 + height
    bbox_area = width * height
    if frame_id not in frame_data:
        frame_data[frame_id] = []
    frame_data[frame_id].append([x1, y1, x2, y2, people_id, bbox_area, conf])

In [ ]:

json_list = []

# Process each frame to calculate overlaps
for frame_id, boxes in frame_data.items():
    bboxes = [[box[0], box[1], box[2], box[3]] for box in boxes]
    overlap_counts = box_overlap_count(bboxes)

    for box in boxes:
        json_obj = {
            "video": video_name,
            "frame": frame_id,
            "people_id": box[4],
            "bbox": [round(box[0], 2), round(box[1], 2), round(box[2], 2), round(box[3], 2)],
            "bbox_area": round(box[5],2),
            "n_overlapping_boxes": overlap_counts,
            "score":  box[6]
        }
        json_list.append(json_obj)

# Convert list to JSON and write to file
json_data = json.dumps(json_list, indent=4)
with open(output_json , 'w') as json_file:
    json_file.write(json_data)